In [1]:
%reset -f  # 清空所有變數

import gc
gc.collect()  # 強制 Python 回收記憶體

Don't know how to reset  #, please run `%reset?` for details
Don't know how to reset  清空所有變數, please run `%reset?` for details


0

In [2]:
# Package
import os
import pandas as pd

In [3]:
# 設定目錄路徑和輸出檔案名稱
date = 250423

directory = os.path.join(".", "summary")
output_dir = os.path.join(".", "combine")
output_file = os.path.join(output_dir, f'combined_{date}.csv')

os.makedirs(output_dir, exist_ok=True)


In [4]:
def process_csv(file_path, skip_header):

    print(f"正在處理檔案: {file_path}")
    try:
        data = pd.read_csv(file_path, encoding='utf-8', skiprows=3)
    except UnicodeDecodeError:
        # 如果 utf-8 讀取失敗，則嘗試使用 ISO-8859-1 編碼
        data = pd.read_csv(file_path, encoding='ISO-8859-1', skiprows=3)

    # 移除全為 NaN 的列
    data_cleaned = data.dropna(how='all')
    return data_cleaned

In [5]:
def modeulate_duty(duty):
    duty_numeric = pd.to_numeric(duty, errors='coerce')
    if pd.notna(duty_numeric):
        return (100 - duty_numeric) / 100
    else:
        return None

def generate_result(mode, function, B_max, frequency, duty_P, duty_N, Hdc, N1, N2, core_loss_Pcv, wave_file_name):
    temp = 25  # 固定溫度值
    return {
        'B_max(mT)': B_max,
        'Frequency(kHz)': frequency,
        'Duty_P': duty_P,
        'Duty_N': duty_N,
        'Hdc(A/m)': Hdc,
        'Temp': temp,
        'N1': N1,
        'N2': N2,
        'Core_Loss(Pcv(kW/m3))': core_loss_Pcv,
        'Wave_file_name': wave_file_name
    }

# *分類AC/DC Sine/Pulse
def classify_and_modeulate(row):
    mode = row['mode']
    function = row['function']
    frequency = pd.to_numeric(row['freq(kHz)'], errors='coerce')
    N1 = pd.to_numeric(row['N1'])
    N2 = pd.to_numeric(row['N2'])
    wave_file_name = row['Wave file name']

    match (mode, function):
        case ('Standard', 'Sine'):
            B_max = pd.to_numeric(row['Bm(T)']) * 1000
            duty_P = -1
            duty_N = -1
            Hdc = 0
            core_loss_Pcv = pd.to_numeric(row['Pcv(kW/m3)'], errors='coerce')
            
        
        case ('Standard', 'Pulse'):
            B_max = pd.to_numeric(row['Bm(T)']) * 1000
            duty_P = 0.5
            duty_N = 0.5
            Hdc = 0
            core_loss_Pcv = pd.to_numeric(row['Pcv(kW/m3)'], errors='coerce')
        
        case ('DC Bias', 'Sine'):
            B_max = (pd.to_numeric(row['delta_B(T)'], errors='coerce') / 2) * 1000
            duty_P = -1
            duty_N = -1
            Hdc = pd.to_numeric(row['Hdc(A/m)'], errors='coerce')
            core_loss_Pcv = pd.to_numeric(row['delta_Pcv(kW/m3)'], errors='coerce')
        
        case ('DC Bias', 'Pulse'):
            B_max = (pd.to_numeric(row['delta_B(T)'], errors='coerce') / 2) * 1000
            duty_P = pd.to_numeric(row['Duty(%)'], errors='coerce') / 100
            duty_N = modeulate_duty(row['Duty(%)'])
            Hdc = pd.to_numeric(row['Hdc(A/m)'], errors='coerce')
            core_loss_Pcv = pd.to_numeric(row['delta_Pcv(kW/m3)'], errors='coerce')
        
        case _:
            return None

    return generate_result(mode, function, B_max, frequency, duty_P, duty_N, Hdc, N1, N2, core_loss_Pcv, wave_file_name)


In [6]:
# 合併檔案並過濾無效資料
def process_all_csv_and_combine(directory, output_file):
    combined_data = []  # 用於儲存所有處理後的資料
    header_included = False

    # 取得目錄下所有 CSV 檔案
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

    for file_name in csv_files:
        file_path = os.path.join(directory, file_name)
        skip_header = header_included
        data_cleaned = process_csv(file_path, skip_header)

        # 應用分類與處理函式
        extracted_data = data_cleaned.apply(classify_and_modeulate,
                                            axis=1).dropna()

        # 轉換成 DataFrame
        extracted_df = pd.DataFrame(extracted_data.tolist())

        # **過濾掉 Wave_file_name 為 "none" 或空白的資料**
        extracted_df = extracted_df[
            (extracted_df["Wave_file_name"].str.lower() != "none")
            &  # 過濾 "none"
            (extracted_df["Wave_file_name"].str.strip() != "")  # 過濾空白
        ]

        # 加入到總數據
        combined_data.append(extracted_df)
        header_included = True

    # 合併所有 DataFrames
    combined_df = pd.concat(combined_data, ignore_index=True)

    # 儲存合併後的資料
    combined_df.to_csv(output_file, index=False)
    print(f"合併資料已儲存到: {output_file}")


In [7]:
# 執行合併所有 CSV 檔案的處理
process_all_csv_and_combine(directory, output_file)


正在處理檔案: .\summary\0CH467160_DC_TRI_100k_1-0mT_N15_D0.6_ALL.csv
正在處理檔案: .\summary\CH467160_AC_Pulse_100k__N10.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N10.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N10_D0.1_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N10_D0.2_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N10_D0.3_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N10_D0.4_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N10_D0.5_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N10_D0.6_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N10_D0.7_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N10_D0.8_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N10_D0.9_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N15_D0.2_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N15_D0.3_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N15_D0.4_ALL.csv
正在處理檔案: .\summary\CH467160_DC_TRI_100k_100mT_N15_D0.5_ALL.csv
正在處理檔案: .\summary\CH467160_D